In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Load the training data into feature matrix, class labels, and event ids:

In [2]:
# predictions 1 is "s", -1 is "b"
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
print(tX.shape)
tX

(250000, 30)


array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

## Preprocessing

In [4]:
from clean_data import *

# save ind_delete to delete indices from test data
tX, ind_delete = update_X(tX, bound_delete=0.8, bound_change=0.05)

In [5]:
from split_expand_data import *

# split data into train and test sets
X_train, y_train, X_test, y_test = split_data(tX, y, split_ratio=0.8)

# store d to later use with real test set
d=7

# create expanded X_train and X_test and normalize
X_train_poly, mu_train_poly, std_train_poly = expand_and_normalize_X(X_train,d)
X_test_poly  = expand_X(X_test,d)
X_test_poly[:,1:]  = (X_test_poly[:,1:]-mu_train_poly)/std_train_poly

# Do your thing crazy machine learning thing here :) ...

### I - Least Squares GD:

In [16]:
from least_squares_GD import *

initial_w = np.random.normal(0, 1e-1, X_train_poly.shape[1])
max_iters = 100
gamma = 0.01

w, loss = least_squares_GD(y_train, X_train_poly, initial_w, max_iters, gamma)
print(loss)

0.4072862969493159


### II Least Squares SGD

In [18]:
# I think this is shitty cause we have 72 params so needs param optimization

from least_squares_SGD import *

initial_w = np.random.normal(0, 1e-1, X_train_poly.shape[1])
max_iters = 100
gamma = 0.001

w, loss = least_squares_SGD(y_train, X_train_poly, initial_w, max_iters, gamma)
print(loss)

0.8353725619951254


### III Least Squares

In [47]:
from least_squares import *

w, loss_analytical = least_squares(y_train, X_train_poly)

print(loss_analytical)

0.3228801100705618


### IV Ridge Regression

### V Logistic Regression

### VI Regularized Logistic Regression

### A)* Cross Validation (do we need it???)

In [11]:
def fold_indices(num_examples,k_fold):
    """
    Splits data indices
    num_examples: total samples in the dataset
    k_fold: number fold of Cross Validation
    
    Returns
    -------
    array of shuffled indices with shape (k_fold, num_examples//k_fold)
    """
    ind = np.arange(num_examples)
    split_size = num_examples//k_fold
    
    # shuffle data
    np.random.shuffle(ind)
    
    k_fold_indices = []
    # Generate k_fold set of indices
    k_fold_indices = [ind[k*split_size:(k+1)*split_size] for k in range(k_fold)]
         
    return np.array(k_fold_indices)

## Predictions based on w

In [105]:
# from gradient_descent import *
from plots import gradient_descent_visualization
import datetime

# Define the parameters of the algorithm.
max_iters = 500
gamma = 0.01

# Initialize weights
w_initial = np.random.normal(0, 1e-1, X_train_poly.shape[1])
print(w)
print(w.shape)

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = gradient_descent(y_train, X_train_poly, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

[ 0.05648531 -0.27337724  0.08573239  0.02571961  0.04590483  0.10738388
 -0.04931987 -0.02135578 -0.0106693  -0.02642069  0.04604404  0.16854969
 -0.14825516 -0.08293466  0.12381492 -0.01210349 -0.05104763  0.07434264
  0.10201889 -0.06826561 -0.19442112  0.01377429  0.19278142 -0.13117964
 -0.0292591   0.08176608  0.10115333 -0.00234948  0.13309682 -0.01474767
  0.10541429  0.1775804  -0.01838765 -0.09570489 -0.06882322 -0.01121606
 -0.08648079 -0.10723669  0.1013338   0.17430125  0.11995028 -0.03722523
 -0.16334417 -0.1229582   0.04695462 -0.13306874  0.06889762  0.05990705
  0.0617199  -0.12687894 -0.05602627  0.02902963 -0.06000911  0.043653
 -0.01680583  0.00688197  0.3132869   0.04430124  0.0277345  -0.03170848
 -0.08332548  0.18111479  0.23710783 -0.09129975 -0.0480382   0.03098816
 -0.12784783 -0.06091947 -0.12412917 -0.1262007 ]
(70,)
Gradient Descent(0/499): loss=0.7616430706831929, w0=0.034209070029780235, w1=-0.09420164271377136
Gradient Descent(1/499): loss=0.650320614857

Gradient Descent(76/499): loss=0.4345262171485151, w0=-0.1522888429895596, w1=-0.028447504239475296
Gradient Descent(77/499): loss=0.4338936206302959, w0=-0.15391555455967212, w1=-0.027945294819744955
Gradient Descent(78/499): loss=0.43327294589528, w0=-0.15552599901408354, w1=-0.02744611745808583
Gradient Descent(79/499): loss=0.43266390050733067, w0=-0.1571203390239508, w1=-0.02694991051109249
Gradient Descent(80/499): loss=0.4320662005109854, w0=-0.1586987356337195, w1=-0.026456614630936154
Gradient Descent(81/499): loss=0.4314795701481471, w0=-0.16026134827739058, w1=-0.025966172657646693
Gradient Descent(82/499): loss=0.4309037415857139, w0=-0.16180833479462492, w1=-0.025478529517031133
Gradient Descent(83/499): loss=0.43033845465364007, w0=-0.16333985144668695, w1=-0.024993632123903507
Gradient Descent(84/499): loss=0.42978345659295797, w0=-0.16485605293222835, w1=-0.024511429290322136
Gradient Descent(85/499): loss=0.4292385018133052, w0=-0.1663570924029144, w1=-0.02403187163855

Gradient Descent(160/499): loss=0.40565763381300557, w0=-0.2450294567845196, w1=0.0065521764399195425
Gradient Descent(161/499): loss=0.40547963966216405, w0=-0.24572876221668438, w1=0.006906030035985363
Gradient Descent(162/499): loss=0.4053037163372843, w0=-0.24642107459452753, w1=0.0072587714182227175
Gradient Descent(163/499): loss=0.40512982824877625, w0=-0.24710646384859225, w1=0.007610408901904087
Gradient Descent(164/499): loss=0.4049579405624642, w0=-0.24778499921011632, w1=0.007960950689818126
Gradient Descent(165/499): loss=0.4047880191811501, w0=-0.24845674921802519, w1=0.008310404874848729
Gradient Descent(166/499): loss=0.4046200307266832, w0=-0.24912178172585497, w1=0.00865877944246875
Gradient Descent(167/499): loss=0.4044539425225213, w0=-0.24978016390860644, w1=0.009006082273151781
Gradient Descent(168/499): loss=0.40428972257676377, w0=-0.2504319622695304, w1=0.009352321144705505
Gradient Descent(169/499): loss=0.40412733956564834, w0=-0.25107724264684517, w1=0.00969

Gradient Descent(245/499): loss=0.395494501528541, w0=-0.285198251484893, w1=0.033311841837599714
Gradient Descent(246/499): loss=0.3954144683815704, w0=-0.2854958689700549, w1=0.03359315691518293
Gradient Descent(247/499): loss=0.39533503379562807, w0=-0.28579051028036523, w1=0.03387381724132747
Gradient Descent(248/499): loss=0.3952561905771039, w0=-0.2860822051775725, w1=0.03415382631654833
Gradient Descent(249/499): loss=0.3951779316488599, w0=-0.28637098312580767, w1=0.034433187613652674
Gradient Descent(250/499): loss=0.3951002500479654, w0=-0.2866568732945605, w1=0.03471190457807567
Gradient Descent(251/499): loss=0.3950231389234812, w0=-0.2869399045616258, w1=0.03498998062821043
Gradient Descent(252/499): loss=0.394946591534293, w0=-0.2872201055160205, w1=0.03526741915573234
Gradient Descent(253/499): loss=0.39487060124699164, w0=-0.28749750446087124, w1=0.035544223525917885
Gradient Descent(254/499): loss=0.39479516153379807, w0=-0.2877721294162735, w1=0.03582039707795806
Grad

Gradient Descent(328/499): loss=0.3903907038405017, w0=-0.30203651645527413, w1=0.054711579546043
Gradient Descent(329/499): loss=0.39034350076795715, w0=-0.30216575129073275, w1=0.054948403173790426
Gradient Descent(330/499): loss=0.39029655279835684, w0=-0.3022936937778368, w1=0.05518479128190612
Gradient Descent(331/499): loss=0.39024985771727455, w0=-0.30242035684006985, w1=0.055420745846964665
Gradient Descent(332/499): loss=0.3902034133370279, w0=-0.30254575327168054, w1=0.05565626883358553
Gradient Descent(333/499): loss=0.39015721749626137, w0=-0.3026698957389751, w1=0.055891362194533514
Gradient Descent(334/499): loss=0.39011126805953344, w0=-0.30279279678159676, w1=0.05612602787081797
Gradient Descent(335/499): loss=0.39006556291691374, w0=-0.3029144688137922, w1=0.05636026779179077
Gradient Descent(336/499): loss=0.3900200999835864, w0=-0.3030349241256657, w1=0.05659408387524321
Gradient Descent(337/499): loss=0.38997487719946045, w0=-0.3031541748844205, w1=0.056827478027501

Gradient Descent(414/499): loss=0.387082162349182, w0=-0.3095148941138233, w1=0.07366649553767904
Gradient Descent(415/499): loss=0.387050862171745, w0=-0.3095693451726968, w1=0.07387204372944642
Gradient Descent(416/499): loss=0.3870196937503957, w0=-0.30962325172098154, w1=0.07407729092070924
Gradient Descent(417/499): loss=0.38698865618741324, w0=-0.3096766192037834, w1=0.07428223833368594
Gradient Descent(418/499): loss=0.3869577485931797, w0=-0.3097294530117573, w1=0.07448688718423932
Gradient Descent(419/499): loss=0.3869269700860833, w0=-0.30978175848165146, w1=0.07469123868191752
Gradient Descent(420/499): loss=0.38689631979242234, w0=-0.30983354089684667, w1=0.07489529402999466
Gradient Descent(421/499): loss=0.3868657968463117, w0=-0.30988480548788994, w1=0.07509905442551108
Gradient Descent(422/499): loss=0.38683540038959136, w0=-0.30993555743302276, w1=0.0753025210593133
Gradient Descent(423/499): loss=0.3868051295717334, w0=-0.3099858018587043, w1=0.07550569511609356
Gradi

In [106]:
p = np.array([1 if x>0 else -1 for x in X_test_poly.dot(gradient_ws[-1].T)])

In [107]:
print('Train Accuracy: %.1f %%' % (np.mean(p == y_test) * 100))

Train Accuracy: 71.2 %


In [29]:
p = np.array([1 if x>0 else -1 for x in X_test_poly.dot(w.T)])

In [30]:
print('Train Accuracy: %.1f %%' % (np.mean(p == y_test) * 100))

Train Accuracy: 77.6 %


## Obtaining results

In [16]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [27]:
tX_test = np.delete(tX_test, ind_delete, axis=1)

tX_test_poly  = expand_X(tX_test,d)
tX_test_poly[:,1:]  = (tX_test_poly[:,1:]-mu_train_poly)/std_train_poly

In [29]:
OUTPUT_PATH = '../data/submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w_analytical, tX_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)